In [1]:
# importing shutil module
import shutil
import pandas as pd
import re
import os
import json
import itertools
import numpy as np
from glob import glob
from pycocotools import mask as cocomask
from PIL import Image


In [2]:
train_res_file = {
    "info": {"contributor": "", "date_created": "", "description": "", "url": "", "version": "", "year": ""},
    "licenses": [{"name": "", "id": 0, "url": ""}],
    "categories": [],
    "images": [],
    "annotations": []
}

valid_res_file = {
    "info": {"contributor": "", "date_created": "", "description": "", "url": "", "version": "", "year": ""},
    "licenses": [{"name": "", "id": 0, "url": ""}],
    "categories": [],
    "images": [],
    "annotations": []
}

In [5]:
categoryObjList = []
categoryNameList = []

lines = []
with open('dataset_labels_super_categories.txt') as f:
    lines = f.readlines()

for line in lines:
    fields = line.split('.')
    category_id = int(fields[0])
    label = fields[1][1:-1]
    catSubCat = label.split('::')
    categoryName = catSubCat[0]
    SubCategoryName = catSubCat[1]
    categoryNameList.append(categoryName)
    categoryObj = {
        "id": category_id, 
        "name":categoryName , 
        "supercategory": SubCategoryName
    }
    categoryObjList.append(categoryObj)
    
train_res_file["categories"] = categoryObjList
valid_res_file["categories"] = categoryObjList  

In [6]:
train_res_file

{'info': {'contributor': '',
  'date_created': '',
  'description': '',
  'url': '',
  'version': '',
  'year': ''},
 'licenses': [{'name': '', 'id': 0, 'url': ''}],
 'categories': [{'id': 0, 'name': 'misc_stuff', 'supercategory': 'coco_thing'},
  {'id': 1, 'name': 'aac_blocks', 'supercategory': 'N/A'},
  {'id': 2, 'name': 'adhesives', 'supercategory': 'N/A'},
  {'id': 3, 'name': 'ahus', 'supercategory': 'N/A'},
  {'id': 4,
   'name': 'aluminium_frames_for_false_ceiling',
   'supercategory': 'N/A'},
  {'id': 5, 'name': 'chiller', 'supercategory': 'N/A'},
  {'id': 6, 'name': 'concrete_mixer_machine', 'supercategory': 'N/A'},
  {'id': 7, 'name': 'concrete_pump_(50%)', 'supercategory': 'N/A'},
  {'id': 8, 'name': 'control_panel', 'supercategory': 'N/A'},
  {'id': 9, 'name': 'cu_piping', 'supercategory': 'N/A'},
  {'id': 10, 'name': 'distribution_transformer', 'supercategory': 'N/A'},
  {'id': 11, 'name': 'dump_truck___tipper_truck', 'supercategory': 'N/A'},
  {'id': 12, 'name': 'emulsion_

In [7]:
def stringToList(input):
    if len(input) == 2:
        return []
    elem_list = input[1 : -1].split(",")
    elem_list_int = [ int(float(elem_list[i])) for i in range(len(elem_list)) ]
    return elem_list_int

In [8]:
categoriesToProcess = categoryNameList[1:49]
categoriesToProcess.append('coco_stuff')


In [9]:
categoriesToProcess

['aac_blocks',
 'adhesives',
 'ahus',
 'aluminium_frames_for_false_ceiling',
 'chiller',
 'concrete_mixer_machine',
 'concrete_pump_(50%)',
 'control_panel',
 'cu_piping',
 'distribution_transformer',
 'dump_truck___tipper_truck',
 'emulsion_paint',
 'enamel_paint',
 'fine_aggregate',
 'fire_buckets',
 'fire_extinguishers',
 'glass_wool',
 'grader',
 'hoist',
 'hollow_concrete_blocks',
 'hot_mix_plant',
 'hydra_crane',
 'interlocked_switched_socket',
 'junction_box',
 'lime',
 'marble',
 'metal_primer',
 'pipe_fittings',
 'rcc_hume_pipes',
 'refrigerant_gas',
 'river_sand',
 'rmc_batching_plant',
 'rmu_units',
 'sanitary_fixtures',
 'skid_steer_loader_(bobcat)',
 'smoke_detectors',
 'split_units',
 'structural_steel_-_channel',
 'switch_boards_and_switches',
 'texture_paint',
 'threaded_rod',
 'transit_mixer',
 'vcb_panel',
 'vitrified_tiles',
 'vrf_units',
 'water_tank',
 'wheel_loader',
 'wood_primer',
 'coco_stuff']

In [12]:
from random import sample

imageIdList = []
imageNameList = []
imagePathList = []
trainImageObjList = []
trainAnnotationObjList = []
validImageObjList = []
validAnnotationObjList = []

sourceRelativePath = '../datasets/'
trainDestinationRelativePath = 'train2017/'
validDestinationRelativePath = 'val2017/'

train_json_file = 'annotations/custom_train.json'
valid_json_file = 'annotations/custom_val.json'
csv_base_path = '../ground_truth_csvs/'


for categoryName in categoriesToProcess:
    df = pd.read_csv(csv_base_path + categoryName + '.csv')
    unique_image_ids = list(df.image_id.unique())
    if categoryName == 'coco_stuff':
        train_image_id_list = unique_image_ids
        valid_image_id_list = []
    else:
        total_image_count = len(unique_image_ids)
        valid_image_count = int(0.1 * total_image_count)
        valid_image_id_list = sample(unique_image_ids, valid_image_count)
        train_image_id_list = list(set(unique_image_ids) - set(valid_image_id_list))
        print(" train_image_id_list: ",train_image_id_list )
        print(" valid_image_id_list: ",valid_image_id_list )
    
    for train_image_id in train_image_id_list:
        records = df[df['image_id'] == train_image_id]
        if records.shape[0] == 0:
            continue
        one_record = records.head(1)
        image_path = one_record['image_path'].unique()[0]
        width = int(one_record['width'].unique()[0])
        height = int(one_record['height'].unique()[0])

        extension = ''
        if '.' in image_path:
            extension = image_path[image_path.index('.'):]
        new_image_name =  categoryName + "_" + str(train_image_id) + extension
        
        imageIdList.append(int(train_image_id))
        imagePathList.append(image_path)
        imageNameList.append(new_image_name)
        sourcePath = sourceRelativePath + image_path
        destinationPath = trainDestinationRelativePath + new_image_name
        
        try:
            shutil.copyfile(sourcePath, destinationPath)
            print("Image copied from sourcePath: ", sourcePath, " to destinationPath: ", destinationPath)
        except:
            print("Error occurred while copying file." + sourcePath + " to: " + destinationPath)
        

        imageObj=  {"id": int(train_image_id),"width": width, "height": height,"file_name": new_image_name, "license": 0,
                    "flickr_url": "",
                    "coco_url": "",
                    "date_captured": 0}
        
        #print("imageObj: train_image_id: ", type(int(train_image_id)), " width: ", type(width), 
        #      " width: ", type(height)," new_image_name ", new_image_name)
        
        #print(" imageObj: ", imageObj)
        
        trainImageObjList.append(imageObj)
        
        for record_no, record in records.iterrows():        
            area = float(record['area'])
            bbox = stringToList(record['bbox'])
            category_id = int(record['category'])

            segmentation = stringToList(record['segment_polygon'])
            annotationId = int(record['id'])
            annotationObj = {"id": annotationId,
                             "image_id": int(train_image_id),
                             "category_id": category_id,
                             "segmentation": [segmentation], 
                             "area": area,
                             "bbox": bbox,
                             "iscrowd": 0 
                             
                            }
            #print("annotationObj: ", annotationObj)
            #print("annotationObj: annotationId: ", type(annotationId),
            #      " train_image_id: ", type(int(train_image_id)), 
            #      " category_id: ", type(category_id)," new_image_name ", type(segmentation),
            #      " area: ", type(area), " bbox: ", type(bbox))


            trainAnnotationObjList.append(annotationObj)
            
    for valid_image_id in valid_image_id_list:
        records = df[df['image_id'] == valid_image_id]
        if records.shape[0] == 0:
            continue
        one_record = records.head(1)
        image_path = one_record['image_path'].unique()[0]
        width = int(one_record['width'].unique()[0])
        height = int(one_record['height'].unique()[0])

        extension = ''
        if '.' in image_path:
            extension = image_path[image_path.index('.'):]
        new_image_name =  categoryName + "_" +  str(valid_image_id) + extension
        imageIdList.append(int(train_image_id))
        imagePathList.append(image_path)
        imageNameList.append(new_image_name)
        sourcePath = sourceRelativePath + image_path
        destinationPath = validDestinationRelativePath + new_image_name
        
        try:
            shutil.copyfile(sourcePath, destinationPath)
            print("Image copied from sourcePath: ", sourcePath, " to destinationPath: ", destinationPath)
        except:
            print("Error occurred while copying file." + sourcePath + " to: " + destinationPath)
        
        imageObj=  {"id": int(valid_image_id), "width": width, "height": height,"file_name": new_image_name, 
                    "license": 0,
                    "flickr_url": "",
                    "coco_url": "",
                    "date_captured": 0}
        #print("imageObj: ", imageObj)
        #print("imageObj: valid_image_id: ", type(int(valid_image_id)), " width: ", type(width), 
        #      " width: ", type(height)," new_image_name ", new_image_name)
        
        validImageObjList.append(imageObj)
        
        for record_no, record in records.iterrows():        
            area = float(record['area'])
            bbox = stringToList(record['bbox'])
            category_id = int(record['category'])

            segmentation = stringToList(record['segment_polygon'])
            annotationId = int(record['id'])
            annotationObj = {"id": annotationId,
                             "image_id": int(valid_image_id),
                             "category_id": category_id,
                             "segmentation": [segmentation], 
                             "area": area,
                             "bbox": bbox,
                             "iscrowd": 0 
                             
                            }
            #print("annotationObj: ", annotationObj)
            #print("annotationObj: annotationId: ", type(annotationId),
            #      " train_image_id: ", type(int(valid_image_id)), 
            #      " category_id: ", type(category_id)," new_image_name ", type(segmentation),
            #      " area: ", type(area), " bbox: ", type(bbox))

            validAnnotationObjList.append(annotationObj)       

        
 

 train_image_id_list:  [104, 105, 107, 108, 114, 116, 117, 118, 120, 123, 125, 127, 129, 131, 134, 139, 140, 141, 143, 147, 149, 151, 152, 154, 157, 158, 159, 160, 161, 162, 167, 171, 174, 178, 179, 180, 186, 187, 189, 190, 196, 197, 200, 201, 203, 205, 214, 225, 228, 235, 238, 241, 243, 244, 245, 246, 247, 251, 254]
 valid_image_id_list:  [110, 198, 122, 239, 128, 209]
Image copied from sourcePath:  ../datasets/aac_blocks/images/img_260.jpg  to destinationPath:  train2017/aac_blocks_104.jpg
Image copied from sourcePath:  ../datasets/aac_blocks/images/img_259.jpg  to destinationPath:  train2017/aac_blocks_105.jpg
Image copied from sourcePath:  ../datasets/aac_blocks/images/img_257.jpg  to destinationPath:  train2017/aac_blocks_107.jpg
Image copied from sourcePath:  ../datasets/aac_blocks/images/img_256.jpg  to destinationPath:  train2017/aac_blocks_108.jpg
Image copied from sourcePath:  ../datasets/aac_blocks/images/img_250.jpeg  to destinationPath:  train2017/aac_blocks_114.jpeg
Image

Image copied from sourcePath:  ../datasets/adhesives/images/img_099.jpg  to destinationPath:  train2017/adhesives_264.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_098.jpg  to destinationPath:  train2017/adhesives_265.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_096.png  to destinationPath:  train2017/adhesives_267.png
Image copied from sourcePath:  ../datasets/adhesives/images/img_095.jpg  to destinationPath:  train2017/adhesives_268.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_094.jpg  to destinationPath:  train2017/adhesives_269.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_093.jpg  to destinationPath:  train2017/adhesives_270.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_091.jpg  to destinationPath:  train2017/adhesives_272.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_090.jpg  to destinationPath:  train2017/adhesives_273.jpg
Image copied from source

Image copied from sourcePath:  ../datasets/adhesives/images/img_009.jpg  to destinationPath:  train2017/adhesives_354.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_008.jpg  to destinationPath:  train2017/adhesives_355.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_006.jpg  to destinationPath:  train2017/adhesives_357.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_005.jpg  to destinationPath:  train2017/adhesives_358.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_004.jpg  to destinationPath:  train2017/adhesives_359.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_002.jpg  to destinationPath:  train2017/adhesives_361.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_001.jpg  to destinationPath:  train2017/adhesives_362.jpg
Image copied from sourcePath:  ../datasets/adhesives/images/img_000.jpg  to destinationPath:  train2017/adhesives_363.jpg
Image copied from source

Image copied from sourcePath:  ../datasets/ahus/images/img_075.jpg  to destinationPath:  train2017/ahus_419.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_074.jpg  to destinationPath:  train2017/ahus_420.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_073.jpg  to destinationPath:  train2017/ahus_421.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_072.jpg  to destinationPath:  train2017/ahus_422.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_071.jpg  to destinationPath:  train2017/ahus_423.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_070.jpg  to destinationPath:  train2017/ahus_424.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_069.jpg  to destinationPath:  train2017/ahus_425.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_143.jpg  to destinationPath:  train2017/ahus_426.jpg
Image copied from sourcePath:  ../datasets/ahus/images/img_142.jpg  to destinationPath:  train2017/ahus_

 train_image_id_list:  [512, 513, 514, 515, 516, 517, 518, 519, 520, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 541, 542, 543, 544, 545, 546, 497, 498, 499, 500, 502, 504, 505, 506, 507, 508, 509, 510, 511]
 valid_image_id_list:  [503, 501, 521, 540, 539]
Image copied from sourcePath:  ../datasets/aluminium_frames_for_false_ceiling/images/index40.jpg  to destinationPath:  train2017/aluminium_frames_for_false_ceiling_512.jpg
Image copied from sourcePath:  ../datasets/aluminium_frames_for_false_ceiling/images/index4.jpg  to destinationPath:  train2017/aluminium_frames_for_false_ceiling_513.jpg
Image copied from sourcePath:  ../datasets/aluminium_frames_for_false_ceiling/images/index39.jpg  to destinationPath:  train2017/aluminium_frames_for_false_ceiling_514.jpg
Image copied from sourcePath:  ../datasets/aluminium_frames_for_false_ceiling/images/index38.jpg  to destinationPath:  train2017/aluminium_frames_for_false_ceiling_515.jpg
Image copied fr

...

Image copied from sourcePath:  ../datasets/coco_stuff/images/000000359781.jpg  to destinationPath:  train2017/coco_stuff_19681.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000152870.jpg  to destinationPath:  train2017/coco_stuff_19682.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000055167.jpg  to destinationPath:  train2017/coco_stuff_19683.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000284762.jpg  to destinationPath:  train2017/coco_stuff_19684.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000162130.jpg  to destinationPath:  train2017/coco_stuff_19685.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000377575.jpg  to destinationPath:  train2017/coco_stuff_19686.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000005037.jpg  to destinationPath:  train2017/coco_stuff_19687.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000147223.jpg  to d

Image copied from sourcePath:  ../datasets/coco_stuff/images/000000313034.jpg  to destinationPath:  train2017/coco_stuff_19744.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000255747.jpg  to destinationPath:  train2017/coco_stuff_19745.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000561223.jpg  to destinationPath:  train2017/coco_stuff_19746.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000563758.jpg  to destinationPath:  train2017/coco_stuff_19747.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000206579.jpg  to destinationPath:  train2017/coco_stuff_19748.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000451155.jpg  to destinationPath:  train2017/coco_stuff_19749.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000150417.jpg  to destinationPath:  train2017/coco_stuff_19750.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000263644.jpg  to d

Image copied from sourcePath:  ../datasets/coco_stuff/images/000000008899.jpg  to destinationPath:  train2017/coco_stuff_19812.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000212895.jpg  to destinationPath:  train2017/coco_stuff_19813.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000285894.jpg  to destinationPath:  train2017/coco_stuff_19814.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000054967.jpg  to destinationPath:  train2017/coco_stuff_19815.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000128372.jpg  to destinationPath:  train2017/coco_stuff_19816.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000301981.jpg  to destinationPath:  train2017/coco_stuff_19817.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000404534.jpg  to destinationPath:  train2017/coco_stuff_19818.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000079565.jpg  to d

Image copied from sourcePath:  ../datasets/coco_stuff/images/000000326128.jpg  to destinationPath:  train2017/coco_stuff_19881.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000214539.jpg  to destinationPath:  train2017/coco_stuff_19882.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000051314.jpg  to destinationPath:  train2017/coco_stuff_19883.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000556873.jpg  to destinationPath:  train2017/coco_stuff_19884.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000417911.jpg  to destinationPath:  train2017/coco_stuff_19885.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000081988.jpg  to destinationPath:  train2017/coco_stuff_19886.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000554266.jpg  to destinationPath:  train2017/coco_stuff_19887.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000008629.jpg  to d

Image copied from sourcePath:  ../datasets/coco_stuff/images/000000252559.jpg  to destinationPath:  train2017/coco_stuff_19953.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000050145.jpg  to destinationPath:  train2017/coco_stuff_19954.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000467848.jpg  to destinationPath:  train2017/coco_stuff_19955.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000572388.jpg  to destinationPath:  train2017/coco_stuff_19956.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000272212.jpg  to destinationPath:  train2017/coco_stuff_19957.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000377000.jpg  to destinationPath:  train2017/coco_stuff_19958.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000198641.jpg  to destinationPath:  train2017/coco_stuff_19959.jpg
Image copied from sourcePath:  ../datasets/coco_stuff/images/000000181421.jpg  to d

In [13]:
train_res_file["images"] = trainImageObjList 
train_res_file["annotations"] = trainAnnotationObjList

valid_res_file["images"] = validImageObjList 
valid_res_file["annotations"] = validAnnotationObjList 

In [14]:
with open(train_json_file, "w") as f:
    json_str = json.dumps(train_res_file)
    f.write(json_str)

In [15]:
with open(valid_json_file, "w") as f:
    json_str = json.dumps(valid_res_file)
    f.write(json_str)

In [16]:
dict = {'image_id':imageIdList,
        'image_path': imagePathList,
        'new_image_name':imageNameList}
  
# creating a dataframe from a dictionary 
df1 = pd.DataFrame(dict)
df1.to_csv('image_mapping.csv', index=False)
